In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime 

In [2]:
df1=pd.read_excel('D:/newdataset/mar.xlsx')
df2=pd.read_excel('D:/newdataset/apr.xlsx')
df3=pd.read_excel('D:/newdataset/may.xlsx')
df4=pd.read_excel('D:/newdataset/jun.xlsx')

In [3]:
df=pd.concat([df1, df2, df3,df4], axis=0)

In [4]:
df['GHI1'] = df['GHI'].shift(1)
df['GHI2'] = df['GHI'].shift(2)
df['GHI3'] = df['GHI'].shift(3)
df['GHI4'] = df['GHI'].shift(4)
df['GHI5'] = df['GHI'].shift(5)
df['GHI6'] = df['GHI'].shift(6)

In [5]:
lower_limit = 7
upper_limit = 18
data = df[(df['Hour'] >= lower_limit) & (df['Hour'] <= upper_limit)]

In [6]:
from sklearn.preprocessing import MinMaxScaler
columns_to_normalize = ['Temperature','GHI','Solar Zenith Angle','Relative Humidity','Cloud Type','Clearsky GHI','GHI1','GHI2','GHI3','GHI4','GHI5','GHI6']
scaler = MinMaxScaler()

data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
data.head()

C:\Users\suraj\AppData\Local\Temp\ipykernel_14428\210008168.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])


,Year,Month,Day,Hour,Minute,Temperature,GHI,Solar Zenith Angle,Relative Humidity,Cloud Type,Clearsky GHI,GHI1,GHI2,GHI3,GHI4,GHI5,GHI6
7,2016,3,1,7,0,0.214900,0.083810,0.854872,0.292062,0.111111,0.083810,0.000000,0.000000,0.000000,0.00000,0.0,0.0
8,2016,3,1,8,0,0.320917,0.278095,0.713085,0.195853,0.000000,0.278095,0.083810,0.000000,0.000000,0.00000,0.0,0.0
9,2016,3,1,9,0,0.429799,0.470476,0.580627,0.140521,0.000000,0.470476,0.278095,0.083810,0.000000,0.00000,0.0,0.0
10,2016,3,1,10,0,0.492837,0.627619,0.465791,0.114455,0.000000,0.627619,0.470476,0.278095,0.083810,0.00000,0.0,0.0
11,2016,3,1,11,0,0.527221,0.709524,0.382861,0.100237,0.000000,0.709524,0.627619,0.470476,0.278095,0.08381,0.0,0.0


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [19]:
X=data[['GHI1','GHI2','GHI3','GHI4','GHI5','GHI6']]
y=data[['GHI']]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)

scores = cross_val_score(rfr, X_test, y_test, cv=10)
y_pred = rfr.predict(X_test)

rmse_rf = mean_squared_error(y_test, y_pred, squared=False)
mae_rf = mean_absolute_error(y_test, y_pred)
r2_rf = r2_score(y_test, y_pred) 

##GHI(maxvalue) = 1050
##GHI(minvalue) = 0


max_rmse = 1050
min_rmse = 0
original_rmse = rmse_rf * (max_rmse - min_rmse) + min_rmse
print('RMSE: ', original_rmse) 

max_mae = 1050
min_mae = 0
original_mae = mae_rf * (max_mae - min_mae) + min_mae
print('MAE:', original_mae) 

print("R-squared: ", r2_rf)

C:\Users\suraj\AppData\Local\Temp\ipykernel_14428\966137838.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfr.fit(X_train, y_train)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

RMSE:  51.097715511748355
MAE: 27.981025690034244
R-squared:  0.9698578247792276


In [21]:
from sklearn.svm import SVR
svr = SVR(kernel='linear')
svr.fit(X_train, y_train)

scores = cross_val_score(svr, X_test, y_test, cv=10)
y_pred = svr.predict(X_test)

rmse_svr = mean_squared_error(y_test, y_pred, squared=False)
mae_svr = mean_absolute_error(y_test, y_pred)
r2_svr = r2_score(y_test, y_pred)

##GHI(maxvalue) = 1050
##GHI(minvalue) = 0

print("Support Vector Regression:")

max_rmse = 1050
min_rmse = 0
original_rmse = rmse_svr * (max_rmse - min_rmse) + min_rmse
print('RMSE: ', original_rmse) 

max_mae = 1050
min_mae = 0
original_mae = mae_svr * (max_mae - min_mae) + min_mae
print('MAE:', original_mae) 

print("R-squared: ", r2_svr)

C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Support Vector Regression:
RMSE:  67.05718993955281
MAE: 52.49272383930666
R-squared:  0.9480886500980035


C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\suraj\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

In [22]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)
rmse_knn = mean_squared_error(y_test, y_pred_knn, squared=False)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

##GHI(maxvalue) = 1050
##GHI(minvalue) = 0

print('K-Nearest Neighbors Regression:')
max_rmse = 1050
min_rmse = 0
original_rmse = rmse_knn * (max_rmse - min_rmse) + min_rmse
print('RMSE: ', original_rmse) 

max_mae = 1050
min_mae = 0
original_mae = mae_knn * (max_mae - min_mae) + min_mae
print('MAE:', original_mae) 

print("R-squared: ", r2_knn)

K-Nearest Neighbors Regression:
RMSE:  52.511436433258545
MAE: 30.241530054644805
R-squared:  0.9681668644220007


In [24]:
from sklearn.neural_network import MLPRegressor

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPRegressor(hidden_layer_sizes=(50, 50, 50, 50, 50), activation='relu', solver='adam', max_iter=100, random_state=42)
mlp.fit(X_train_scaled, y_train)
y_pred = mlp.predict(X_test_scaled)

rmse_ann = mean_squared_error(y_test, y_pred, squared=False)
mae_ann = mean_absolute_error(y_test, y_pred)
r2_ann = r2_score(y_test, y_pred)

max_value = 1050
min_value = 0
original_rmse = rmse_ann * (max_value - min_value) + min_value
original_mae = mae_ann * (max_value - min_value) + min_value

print('RMSE:', original_rmse)
print('MAE:', original_mae)
print('R-squared:', r2_ann)

C:\Users\suraj\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1617: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RMSE: 53.32248874100287
MAE: 33.32158571191089
R-squared: 0.9671759289155891
